In [1]:
import pandas as pd
df=pd.read_csv("BTCUSD_Candlestick_15_M_ASK_05.08.2019-29.04.2022.csv")

In [2]:
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [3]:
import pandas_ta as ta
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df["EMA"]=ta.ema(df.Close, length=100)

In [4]:
emasignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

In [5]:
VWAPsignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.VWAP[i]:
            dnt=0
        if df.Low[i]<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

In [16]:
def TotalEMASignal(l):
    myclosedistance = 100
    if (df.EMASignal[l]==2  
        and min(abs(df.EMA[l]-df.High[l]),abs(df.EMA[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.EMASignal[l]==1 
        and min(abs(df.EMA[l]-df.High[l]),abs(df.EMA[l]-df.Low[l]))<=myclosedistance):
            return 1

def TotalVWAPSignal(l):
    myclosedistance = 100
    if (df.VWAPSignal[l]==2 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.VWAPSignal[l]==1 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 1        

# Change signal Here VWAP and EMA
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalVWAPSignal(row)
df['TotalSignal'] = TotSignal

In [17]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-3
    elif x['TotalSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[150:400]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, line=dict(color='blue', width=1), name="VWAP")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [19]:
dfpl = df[:].copy()
# dfpl.reset_index(inplace=True)
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=5)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [25]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 0.8*self.data.ATR[-1]
        TPSLRatio = 2.5

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

Start                     2019-08-05 00:00:00
End                       2022-04-29 23:45:00
Duration                    998 days 23:45:00
Exposure Time [%]                     23.5429
Equity Final [$]                 573803.00066
Equity Peak [$]                2416278.510181
Return [%]                         473.803001
Buy & Hold Return [%]              248.678633
Return (Ann.) [%]                   94.917708
Volatility (Ann.) [%]              147.530771
Sharpe Ratio                         0.643376
Sortino Ratio                        2.678891
Calmar Ratio                         1.227344
Max. Drawdown [%]                  -77.335885
Avg. Drawdown [%]                   -2.771234
Max. Drawdown Duration      394 days 23:00:00
Avg. Drawdown Duration        3 days 06:36:00
# Trades                                 3852
Win Rate [%]                         31.25649
Best Trade [%]                       5.493236
Worst Trade [%]                     -5.548622
Avg. Trade [%]                    

In [21]:
bt.plot()

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:119: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:141: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



Row(id='3097', ...)

In [13]:
class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.EMA[-1]
            tp1 = self.data.Close[-1] + (self.data.Close[-1]-self.data.EMA[-1])*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.EMA[-1]
            tp1 = self.data.Close[-1] - (self.data.EMA[-1]-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

Start                     2019-08-05 00:00:00
End                       2022-04-29 23:45:00
Duration                    998 days 23:45:00
Exposure Time [%]                   26.851294
Equity Final [$]                 92736.622288
Equity Peak [$]                 126391.195091
Return [%]                          -7.263378
Buy & Hold Return [%]              248.678633
Return (Ann.) [%]                   -2.873526
Volatility (Ann.) [%]               68.918493
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -56.249186
Avg. Drawdown [%]                   -8.531405
Max. Drawdown Duration      512 days 00:45:00
Avg. Drawdown Duration       34 days 08:40:00
# Trades                                 2330
Win Rate [%]                        49.484979
Best Trade [%]                      10.415421
Worst Trade [%]                    -12.301706
Avg. Trade [%]                    

In [14]:
bt.plot()

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:119: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:141: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



Row(id='2286', ...)

In [22]:
class MyStrat(Strategy):
    initsize = 0.5
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.VWAP[-1]
            tp1 = self.data.Close[-1] + (self.data.Close[-1]-self.data.VWAP[-1])*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.VWAP[-1]
            tp1 = self.data.Close[-1] - (self.data.VWAP[-1]-self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=0.0000)
stat = bt.run()
stat

Start                     2019-08-05 00:00:00
End                       2022-04-29 23:45:00
Duration                    998 days 23:45:00
Exposure Time [%]                   25.286337
Equity Final [$]                300376.062844
Equity Peak [$]                 919685.983161
Return [%]                         200.376063
Buy & Hold Return [%]              248.678633
Return (Ann.) [%]                   50.989698
Volatility (Ann.) [%]              122.245643
Sharpe Ratio                         0.417109
Sortino Ratio                        1.232566
Calmar Ratio                         0.752008
Max. Drawdown [%]                  -67.804748
Avg. Drawdown [%]                   -3.364743
Max. Drawdown Duration      505 days 08:45:00
Avg. Drawdown Duration        5 days 16:54:00
# Trades                                 2724
Win Rate [%]                        50.440529
Best Trade [%]                      10.602446
Worst Trade [%]                     -8.961749
Avg. Trade [%]                    

In [23]:
bt.plot()

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:119: UserWarning:

Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`

D:\ProgramFiles\Anaconda3\lib\site-packages\backtesting\_plotting.py:141: FutureWarning:

casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.



Row(id='3908', ...)